# NumPy Broadcasting

## Introduction

The term **broadcasting** describes how NumPy treats arrays with different shapes during arithmetic operations.

- If the shapes are not the same, NumPy tries to “stretch” the smaller array across the larger one so they can work together.

- This avoids making unnecessary copies of data and makes operations fast and memory efficient.

## Why Broadcasting? 
let's say if i want to add a scaler value to each elemnt of an array:

One way is to manually create another array **with the same shape**, filled with that scalar, and then add them together.  
For example:

In [3]:
import numpy as np

a = np.array([1, 2, 3, 4])
b = np.array([2, 2, 2, 2])   # repeating 2 
print(a + b)

[3 4 5 6]


If `a` had 1 million elements, then `b` would also need 1 million elements, all holding the same number — it **wastes memory**.

**With broadcasting**, NumPy avoids creating that second array

In [6]:
a = np.array([1, 2, 3, 4])
b = 2 
print(a + b)   


[3 4 5 6]


The result is the same as in the earlier example where `b` was an array of `[2, 2, 2, 2]`.  

You can think of **broadcasting** as NumPy **"stretching"** the scalar `2` into an array that matches the shape of `a`. Conceptually, it looks like this:

```
[1, 2, 3, 4] .* [2, 2, 2, 2]
= [6, 7, 8, 9]
```

But this stretching is only a **mental model**.  

In reality, NumPy is smart enough **not** to actually create `[2, 2, 2, 2]` in memory.  
Instead, it keeps the single scalar value `2` and applies it **element by element** directly in efficient C code.  

This way, broadcasting is both:  
- **Memory efficient** (no duplicate arrays are created)  
- **Computationally efficient** (fast vectorized operations instead of slow Python loops)

---


# General Broadcasting Rules

Broadcasting decides how NumPy can make arrays with **different shapes** work together.  
Here’s how it works:

1. **Compare shapes from right to left**   
2. Two dimensions are compatible if:  
   - They are the same size, OR  
   - One of them is `1` (NumPy will stretch it).  
3. If the dimensions don’t match these rules → error.  

In [9]:
a1 = np.array([1, 2, 3])
b1 = np.array([[10], [20], [30]])
print(f"shape pf a1 = {a1.shape}, shape of b1 = {b2.shape}")
print(a1 + b1)

NameError: name 'b2' is not defined

`a` has shape (3,) >>> this means it’s 1D with 3 elements.

`b` has shape (3,1) >>> this means it’s 2D: 3 rows, 1 column.

   **pad the smaller shape**
   
Broadcasting always compares dimensions **from the right**.
BUT if shapes **don’t** have **the same number of dimensions**, NumPy pretends **the missing dimensions** are **1**.

So:

Shape of `a (3,)` >>> treat it as `(1,3)` (added a leading 1)

Shape of `b (3,1)`

Now they both have 2 dimensions.

   ***Compare dimension by dimension (from right to left)***
   
Rightmost: 3 vs 1 >>> OK (rule: one is 1 >>> stretch the 1 to 3)

Next: 1 vs 3 >>> OK (again, one is 1 >>> stretch the 1 to 3)

   ***Result shape***
   
After stretching:
`a (1,3)` becomes `(3,3)` (its single row repeats 3 times)

`b (3,1)` becomes `(3,3)` (its single column repeats 3 times)

So the result shape = (3,3):
 ```
[[11 12 13]
 [21 22 23]
 [31 32 33]]
```

REMEMBER:
- NumPy doesn’t actually make copies, it just pretends they were stretched.

- That’s why it’s fast and memory efficient.

In [17]:
#let's take another example of scaler broadcasting
a2 = np.array([[2, 1],
               [3, 2],
               [1, 1]])
b2 = 2
print(f"shape of a2 = {a2.shape}")
print(a2 + b2)

shape of a2 = (3, 2)
[[4 3]
 [5 4]
 [3 3]]


- Scalar `2` is treated as shape `()`

- Stretched to `(3, 2)` during the operation

## Something else in my mind... Scalars

Since we’re talking about broadcasting, you might wonder:  
  *what exactly is a scalar?*  
  *is there a difference between a Python scalar and a NumPy scalar?*  

Let’s explore this:


In [19]:
#python scalar
py_scalar = 2
print("Python scalar:", py_scalar, ">>> type:", type(py_scalar))

#numPy scalar (0-dimensional array)
np_scalar = np.array(2)
print("NumPy scalar:", np_scalar, ">>> type:", type(np_scalar))
print("Shape of NumPy scalar:", np_scalar.shape)       #just ()


Python scalar: 2 >>> type: <class 'int'>
NumPy scalar: 2 >>> type: <class 'numpy.ndarray'>
Shape of NumPy scalar: ()


- A **Python scalar** is just a plain integer (`int`, `float`, etc.).  
- A **NumPy scalar** looks like a single number, but technically it’s a **0-dimensional array** (`shape = ()`).  


In [21]:
#how they both behave in broadcasting?
arr = np.array([1, 2, 3])

print("Array:", arr)
print("Add Python scalar:", arr + py_scalar)
print("Add NumPy scalar:", arr + np_scalar)


Array: [1 2 3]
Add Python scalar: [3 4 5]
Add NumPy scalar: [3 4 5]


- The results are the same in both!!

  
In broadcasting, **both Python scalars and NumPy scalars behave the same**.  

- NumPy treats a scalar as having `shape = ()`.  
- When combined with another array, it conceptually "expands" to match the shape of that array.

---
lets's go back to broadcasting and take an example of **incompatible shapes**

In [23]:
a3 = np.ones((3, 2))   # shape (3,2)
b3 = np.ones((4, 2))   # shape (4,2)

print(a3 + b3)

ValueError: operands could not be broadcast together with shapes (3,2) (4,2) 

a3 has shape `(3, 2)` ans b3 has shape `(4, 2)`

- First, compare from right: 2 vs 2 >>> OK
- Next: 3 vs 4 >>> not equal, not 1 >>> Error
  
And will give `ValueError: operands could not be broadcast together with shapes (3,2) (4,2)`

---

### let's take another example: Scaling RGB image
Imagine you have an image stored as a `256 x 256 x 3` NumPy array:
- `256 x 256` → height and width of the image  
- `3` → RGB channels (Red, Green, Blue)  

Now suppose we want to scale each color channel by a different factor.  
We can do this by multiplying the image with a **1D array of length 3**.

In [25]:
#rgb image
image = np.random.rand(256, 256, 3)  

# Scale factors
scale = np.array([0.5, 1.0, 1.5])  

# Broadcasting in action
scaled_image = image * scale  

print("Image shape:", image.shape)
print("Scale shape:", scale.shape)
print("Result shape:", scaled_image.shape)

Image shape: (256, 256, 3)
Scale shape: (3,)
Result shape: (256, 256, 3)


### Again how it works!
- `image` has shape `(256, 256, 3)`  
- `scale` has shape `(3,)`
  
- Shape `(3,)` means it’s a 1D array with 3 elements.

When compared to `(256, 256, 3)`, NumPy will pad on the left with 1s until the number of dimensions matches.

So (3,) → (1, 3) → (1, 1, 3)

Now they line up:
  - (256, 256, 3)
  - ( 1, 1, 3)
    
- NumPy compares shapes from the **trailing dimensions** -from right to left-:  
  - `3` (from image) vs `3` (from scale) → compatible  
- The other dimensions of `scale` are treated as size `1` and stretched.  

Final result has shape `(256, 256, 3)`.

---

In [27]:
#higher dimensions
a4 = np.zeros((8, 1, 6, 1))
b4 = np.zeros((7, 1, 5))

result = a4 + b4
print("Shape of a4:", a4.shape)
print("Shape of b4:", b4.shape)
print("Shape of result:", result.shape)


Shape of a4: (8, 1, 6, 1)
Shape of b4: (7, 1, 5)
Shape of result: (8, 7, 6, 5)


### Another example- Higher-dimensional broadcasting

- `a4` has shape `(8, 1, 6, 1)`  
- `b4` has shape `(    7, 1, 5)`  

According to the broadcasting rules:  
- Align trailing dimensions → `(8, 1, 6, 1)` vs `(   7, 1, 5)`  
- Compare dimension by dimension:  
  - `1` vs `5` → stretch the `1` to `5`  
  - `6` vs `1` → stretch the `1` to `6`  
  - `1` vs `7` → stretch the `1` to `7`  
  - `8` vs missing → treat as `8 vs 1`, so stretch to `8`  

Resulting shape: `(8, 7, 6, 5)`